In [1]:
import numpy as np
import pandas as pd
import datetime
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

import matplotlib.pyplot as plt
%matplotlib inline

import sys, os, gc, types
import time
from subprocess import check_output

In [2]:
root_paths = [
    "/data/kaggle-wikipedia/",
    "/Users/jiayou/Dropbox/JuanCode/Kaggle/Wikipedia/",
    "/Users/jiayou/Dropbox/Documents/JuanCode/Kaggle/Wikipedia/"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(check_output(["ls", root]).decode("utf8"))

EDA data2.ipynb
EDA.ipynb
Key Modify.ipynb
LightGBM Sub5 Classifier.ipynb
LightGBM Training.ipynb
Public Leaderboard.ipynb
README.md
RNN Overfit.ipynb
RNN Submission.ipynb
RNN.ipynb
Running Median.ipynb
data
data2
tf-data



In [3]:
key1 = pd.read_pickle(root + 'data/key_1_modified.pkl')

In [4]:
start_date = '2017-01-01'
period = 60
date_range = pd.date_range(start_date, periods=period, freq='D')
date_range = date_range.strftime('%Y-%m-%d').tolist()

In [5]:
train2 = pd.read_csv(root + 'data2/train_2.csv')
train2 = train2.loc[:,['Page'] + date_range]
train2.sort_values(by='Page', inplace=True)
train2.reset_index(inplace=True, drop=True)
train2 = train2.melt(id_vars=['Page'], var_name='date', value_name='y_true')

In [13]:
name = 'gb5-r1'
sub5_name = 'sub5-r1'

In [ ]:
def get_submission(name):
    submission = pd.read_csv(root + 'data/test_prediction.{}.csv'.format(name))
    submission = submission.merge(key1, how='left', on='Id')
    submission.rename(columns={"Visits": "y_pred"}, inplace=True)
    return submission

def get_submission_with_sub5(name, sub5_name):
    submission = pd.read_csv(root + 'data/test_prediction.{}.csv'.format(name))
    sub5 = pd.read_csv(root + 'data/sub5_prediction.{}.csv'.format(sub5_name))
    submission.Visits = np.where(submission.Visits.values <= 5, sub5.Visits.values, submission.Visits.values)
    
    submission = submission.merge(key1, how='left', on='Id')
    submission.rename(columns={"Visits": "y_pred"}, inplace=True)
    return submission

In [14]:
submission = get_submission(name)
# submission = get_submission_with_sub5(name, sub5_name)

In [15]:
result = train2.merge(submission, on=['Page', 'date'], how='left')

In [16]:
result.dropna(axis=0, how='any', inplace=True)

In [17]:
def SMAPE(y_pred, y_true):
    loss = np.mean(np.abs(y_true - y_pred) / np.maximum(1e-6, (np.abs(y_true) + np.abs(y_pred)))) * 200
    return loss

In [18]:
score = SMAPE(result.y_pred.values, result.y_true.values)

In [19]:
score

45.16190137271969